# Instructions

The following code was designed in order to implement batch processing of location tracking.  Currently, for this to work, all videos must take the same settings, including the defined regions of interest.  To confirm that regions of interest are consistent across videos, for each video an overlay of the reference frame, the regions of interest and the animal trace are produced.  Moreover, the reference frame is currently required to be generated by taking the median of each individual video.  In addition to saving frame by frame location, distance travelled, and whether the animal is in each ROI in separate csv files, bins can also be defined for summarizing videos (e.g. minute by minute).  All summary information will be saved in a single file.

### Package Requirements
Please see instructions under repository README for package requirements and install instructions.

# 1. Load Necessary Packages
The following code loads neccessary packages and need not be changed by the user.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import LocationTracking_Functions as lt
import holoviews as hv

# 2. User Defines Batch Processing Directory and Settings
Below, options are set by user for batch processing of videos.  If you are unfamiliar with these settings, please see LocationTracking_Individual.ipynb.
***Windows Users:*** Place an 'r' in front directory path (e.g. r"zp\Videos") to avoid mishandling of forward slashes.

In [ ]:
#define video paramaters
video_dict = {
    'dpath' : "/Users/me/MyVideos", 
    'ftype' : "avi", 
    'fps' : 30, 
    'start' : 0, 
    'end' : None 
}
stretch = dict(width = 1, height = 1)


#define parameters for location tracking
tracking_params = {
    'loc_thresh' : 99,
    'use_window' : True, 
    'window_size' : 100, 
    'window_weight' : 1, 
    'method' : 'abs',
    'angle_params' : None 
}

#Set paramaters for tracking orientation
#Be sure to comment this section out or set
#tracking_params['angle_params']=None if you do
#not want to track orientation
angle_params = {
    'angle_length' : 70,
    'angle_arc' : 30,
    'angle_mindist' : 1.5,
    'angle_start' : 0
}
tracking_params['angle_params'] = angle_params


#set bin_dict
#set bin_dict = None if only overall session average is desired
bin_dict = {
    1: (0, 500),
    2: (500, 1000),
    3: (500,1000)
}


#set region_names and object_names
#set region_names and object_names to None if no ROIs are needed
region_names=['Left','Right']
object_names=['o1','o2']


#code below loads folder with files.  Needn't be changed.
video_dict = lt.Batch_LoadFiles(video_dict)
video_dict['FileNames']

# 3. (Optional) Crop Image if Desired
To crop video frame, after running code below, select box selection tool below image (square with a plus sign).  To start drawing region to be included in analyis, double click image.  Double click again to finalize region.  If you decide to change region, it is best to rerun this cell and subsequent steps.  Note that this is done based upon first video in folder.

In [ ]:
%%output size=100 

image,crop,video_dict=lt.LoadAndCrop(video_dict,stretch,cropmethod='Box',fstfile=True)
image

# 4. (Optional) Define Regions of Interest

After running cell below, draw regions of interest on presented image in the order you provided them.  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell.  Note that this is done based upon first video in folder.

In [ ]:
%%output size=100

reference,image = lt.Reference(video_dict, stretch, fstfile=True, num_frames=100,
                         crop=crop if 'crop' in locals() else None, ) 
plot,roi_stream = lt.ROI_plot(reference,region_names,stretch) 
plot

# 5. (Optional) Define Objects of Interest

After running cell below, draw objects of interest on presented image in the order you provided them.  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell.

In [ ]:
%%output size=100

reference,image = lt.Reference(video_dict, stretch, fstfile=True, num_frames=100,
                         crop=crop if 'crop' in locals() else None, ) 
plot,object_stream = lt.ROI_plot(reference,object_names,stretch)
plot

---
# 6. (Optional) Define Scale for Distance Calculations

### 6a. Select two points of known distance

After running cell below, click on any two points and the distance between them, in pixel units, will be presented/returned. Will be used to convert pixel distance to other scale. Note that once drawn, points can be dragged or you can click again.

In [ ]:
%%output size = 100
reference,image = lt.Reference(video_dict, fstfile=True, num_frames=100,
                         crop=crop if 'crop' in locals() else None, ) 
dist_plot, dist = lt.DistanceTool(reference, stretch)
dist_plot

### 6b. Define real-world distance between points
Below, set the distance between the points selected below, and the scale. Note that scale can be any desired text.

In [ ]:
scale_dict = {
    'distance' : 50, 
    'scale' : 'cm' 
}

# 7. Perform Batch Processing and Display Traces from Each Session

The code below will save frame by frame data for each video file in its own csv. Binned summary information will be saved in a single file entitled 'BatchSummary.csv'. Additionally, the reference frame for each session will be displayed, along with a trace of the regions of interest (if supplied), and a trace of the animal's location across the session.

In [ ]:
%%opts Layout [shared_axes=False] 
%%output size=100 

summary, images = lt.Batch_Process(video_dict,tracking_params,bin_dict,region_names,object_names,stretch,
                          scale_dict=scale_dict if 'scale_dict' in locals() else None,
                          dist=dist if 'dist' in locals() else None,
                          crop=crop if 'crop' in locals() else None,
                          roi_stream=roi_stream if 'roi_stream' in locals() else None,
                          object_stream=object_stream if 'object_stream' in locals() else None)
images.cols(2)